# OS Credential Dumping: DCSync (T1003.006)
Adversaries may attempt to access credentials and other sensitive information by abusing a Windows Domain Controller's application programming interface (API) to simulate the replication process from a remote domain controller using a technique called DCSync.

Members of the Administrators, Domain Admins, and Enterprise Admin groups or computer accounts on the domain controller are able to run DCSync to pull password data from Active Directory, which may include current and historical hashes of potentially useful accounts such as KRBTGT and Administrators. The hashes can then in turn be used to create a Golden Ticket for use in Pass the Ticket or change an account's password as noted in Account Manipulation.

DCSync functionality has been included in the "lsadump" module in Mimikatz. Lsadump also includes NetSync, which performs DCSync over a legacy replication protocol.

## Where has this exploit been used before?
| ID | Name | Description |
| --- | --- | --- |
| G0016 | APT29 | APT29 leveraged privileged accounts to replicate directory service data with domain controllers. |
| S0002 | Mimikatz | Mimikatz performs credential dumping to obtain account and password information useful in gaining access to additional systems and enterprise network resources. It contains functionality to acquire information about credentials in many ways, including from DCSync/NetSync. |
| G0116 | Operation Wocao | Operation Wocao has used Mimikatz's DCSync to dump credentials from the memory of the targeted system. | 

**Source**: [MITRE ATT&CK T1003.006](https://attack.mitre.org/techniques/T1003/006/)

## Technical Context
Active Directory replication is the process by which the changes that originate on one domain controller are automatically transferred to other domain controllers that store the same data. Active Directory data takes the form of objects that have properties, or attributes. Each object is an instance of an object class, and object classes and their respective attributes are defined in the Active Directory schema. The values of the attributes define the object, and a change to a value of an attribute must be transferred from the domain controller on which it occurs to every other domain controller that stores a replica of that object.

## Offensive Tradecraft

An adversary can abuse this model and request information about a specific account via the replication request. This is done from an account with sufficient permissions (usually domain admin level) to perform that request. Usually the accounts performing replication operations in a domain are computer accounts (i.e dcaccount$). Therefore, it might be abnormal to see other non-dc-accounts doing it.

The following access rights / permissions are needed for the replication request according to the domain functional level:

| Control access right symbol | Identifying GUID used in ACE |
| --- | --- | 
| DS-Replication-Get-Changes | 1131f6aa-9c07-11d1-f79f-00c04fc2dcd2 |
| DS-Replication-Get-Changes-All | 1131f6ad-9c07-11d1-f79f-00c04fc2dcd2 |
| DS-Replication-Get-Changes-In-Filtered-Set | 89e95b76-444d-4c62-991a-0facbeda640c |

**Source**: [Threat Hunters Playbook - Active Directory Replication from Non-Domain-Controller Accounts](https://threathunterplaybook.com/notebooks/windows/06_credential_access/WIN-180815210510.html)

## Dataset Description
This dataset represents adversaries abusing Active Directory Replication services to retrieve secret domain data (i.e. NTLM hashes) from domain accounts.

**Source**: https://securitydatasets.com/notebooks/atomic/windows/credential_access/SDWIN-190301174830.html

In [7]:
# Read the dataset. The dataset is a json zip file that needs to be uncompressed and read into pandas
import requests
import pandas
from zipfile import ZipFile
from io import BytesIO
from pandas.io import json

pandas.set_option('display.max_rows', 200)

url = "https://raw.githubusercontent.com/OTRF/Security-Datasets/master/datasets/atomic/windows/credential_access/host/empire_dcsync_dcerpc_drsuapi_DsGetNCChanges.zip"
zipFileRequest = requests.get(url)
zipFile = ZipFile(BytesIO(zipFileRequest.content))
datasetJSONPath = zipFile.extract(zipFile.namelist()[0])
df = json.read_json(path_or_buf=datasetJSONPath, lines=True)
df.head().T

,0,1,2,3,4
Keywords,36028797018963968,36028797018963968,36028797018963968,0,36028797018963968
SeverityValue,2,2,2,2,2
EventReceivedTime,2020-09-21 18:58:30,2020-09-21 18:58:30,2020-09-21 18:58:30,2020-09-21 18:58:30,2020-09-21 18:58:30
EventID,800,800,800,4103,800
ExecutionProcessID,0,0,0,7456,0
Channel,Windows PowerShell,Windows PowerShell,Windows PowerShell,Microsoft-Windows-PowerShell/Operational,Windows PowerShell
host,wec.internal.cloudapp.net,wec.internal.cloudapp.net,wec.internal.cloudapp.net,wec.internal.cloudapp.net,wec.internal.cloudapp.net
ThreadID,0,0,0,840,0
EventType,INFO,INFO,INFO,INFO,INFO
Opcode,Info,Info,Info,To be used when operation is just executing a ...,Info


In [12]:
df.EventID.unique()

array([ 800, 4103,   10,    5,   12, 4689,   13, 5158, 5156, 4690, 4658,
       4656, 4663, 4672, 4624, 4627, 4634,  600,  400,    7,   17,   18,
          1, 4688,    9,   11, 5857,   22, 4703, 4670, 4673, 4769, 4662,
          3, 5140, 5145,   23])